# Lesson  3 Building the Tutor Service Layer

Absolutely! Here’s your text, **converted to markdown** with headings, code blocks, and clean formatting for readability:

---

# Building the Tutor Service Layer

In the previous lesson, we explored the `SessionManager` class, which plays a crucial role in managing tutoring session data within our application. Now, we will take the next step in our journey by building the **Tutor Service Layer**. This layer is essential for integrating the DeepSeek language model with tutoring sessions, allowing us to process student queries and generate tailored explanations. By the end of this lesson, you will understand how to set up the `TutorService` class, create tutoring sessions, and process academic questions using DeepSeek models via the OpenAI SDK.

The service layer acts as a bridge between the model layer, where data is managed, and the AI model, which generates educational responses. It is responsible for orchestrating the flow of data and ensuring that student interactions are handled effectively. Let's dive into the details of setting up this important component.

---

## Setting Up the TutorService Class

The `TutorService` class is the heart of our service layer. It is responsible for managing tutoring sessions and interacting with the DeepSeek model to generate educational responses. To begin, we need to set up the class and its components.

First, we import the necessary modules, including the `SessionManager` from our previous lesson and the OpenAI client (which we'll use to access DeepSeek models). We also use the `uuid` module to generate unique session IDs. Here’s how the class is initialized:

```python
import uuid
from openai import OpenAI
from models.session import SessionManager

class TutorService:
    def __init__(self):
        self.session_manager = SessionManager()
        self.deepseek_client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')
```

In this setup, we instantiate `SessionManager` to manage tutoring data, initialize the OpenAI client for DeepSeek model access, and load the `system_prompt` using the `load_system_prompt` method, which we'll discuss next.

---

## Loading the System Prompt

The system prompt is a crucial component that guides the tutor AI's responses. It provides context and instructions for the AI, ensuring that it behaves like an effective educational assistant. In this section, we'll implement the `load_system_prompt` method to load the prompt from a file.

```python
def load_system_prompt(self, file_path: str) -> str:
    """Load the tutor system prompt from file."""
    try:
        with open(file_path, 'r') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading system prompt: {e}")
        return "You are a helpful tutor."
```

This method attempts to read the system prompt from a specified file path. If successful, it returns the prompt as a string. In case of an error, it prints an error message and returns a default prompt. This ensures that the application can continue functioning even if the file is missing or corrupted.

---

## Creating a New Tutoring Session

Creating a new tutoring session is a fundamental task of the `TutorService`. The `create_session` method is responsible for generating a unique session ID and initializing a tutoring session using the `SessionManager`.

```python
def create_session(self, student_id: str) -> str:
    """Create a new tutoring session."""
    session_id = str(uuid.uuid4())
    self.session_manager.create_session(student_id, session_id, self.system_prompt)
    return session_id
```

In this method, we generate a unique `session_id` using the `uuid` module. We then call the `create_session` method of `SessionManager`, passing the `student_id`, `session_id`, and `system_prompt`. This initializes a new tutoring session, which is ready to receive student queries.

---

## Processing Student Queries

The `process_query` method is where the educational magic happens. It processes student questions, interacts with the DeepSeek model to generate tutoring explanations, and updates the session history. Below, we outline the steps involved in this process, followed by the corresponding code implementation:

* Retrieve the session using `get_session`, and raise an error if the session is not found.
* Add the student's query to the session history.
* Send the conversation, including the system prompt and all previous exchanges, to the DeepSeek model to generate a response.
* Add the tutor's explanation to the session history and return it to the student.
* Handle any errors with the AI client gracefully.

```python
def process_query(self, student_id: str, session_id: str, query: str) -> str:
    """Process a student query and provide a tutoring explanation."""
    
    # Step 1: Retrieve the session
    session = self.session_manager.get_session(student_id, session_id)
    if not session:
        raise ValueError("Session not found")
    
    # Step 2: Add student's query to session
    self.session_manager.add_message(student_id, session_id, "user", query)
    
    try:
        # Step 3: Get tutor response
        conversation = self.session_manager.get_conversation(student_id, session_id)
        
        response = self.deepseek_client.chat.completions.create(
            model="deepseek-ai/DeepSeek-V3",
            messages=conversation,
            temperature=0.6,
            max_tokens=500
        )
        
        tutor_response = response.choices[0].message.content.strip()
        
        # Step 4: Add tutor's explanation to session history
        self.session_manager.add_message(student_id, session_id, "assistant", tutor_response)
        
        return tutor_response
        
    except Exception as e:
        # Step 5: Handle errors
        raise RuntimeError(f"Error getting tutor response: {str(e)}")
```

In the context of a personal tutor, we configure our DeepSeek model with specific parameters to optimize its educational performance. The temperature is set to `0.6`, which balances accuracy and creativity in the tutor's explanations, ensuring they are both informative and engaging. The `max_tokens` is set to `500`, allowing the model to provide detailed educational content without overwhelming the student, thus maintaining an effective learning experience.

---

## Example: Simulating a Tutoring Session

Let's see the `TutorService` in action by simulating a tutoring session. We'll create a script to initialize a tutoring session and process a student's academic query.

```python
from services.tutor_service import TutorService

# Initialize the tutor service
tutor_service = TutorService()

# Simulate a student ID
student_id = "student123"

# Create a new tutoring session
session_id = tutor_service.create_session(student_id)
print(f"Tutoring session created with ID: {session_id}")

# Simulate sending a tutoring query
student_query = "Can you explain the principles of supply and demand in economics?"

try:
    tutor_response = tutor_service.process_query(student_id, session_id, student_query)
    print(f"Tutor Response: {tutor_response}")
except Exception as e:
    print(f"Error: {e}")
```

**Plain text output:**

```
Tutoring session created with ID: 01a17870-8a4f-4b6f-a3ce-f04e1136d597
Tutor Response: Supply and demand are fundamental principles in economics that describe how prices are determined in a market economy. Let me explain each concept and how they interact:

1. Supply: This refers to the quantity of a good or service that producers are willing and able to offer for sale at various price points. 
   - The law of supply states that as the price of a good increases, the quantity supplied also increases (and vice versa).
   - This creates an upward-sloping supply curve when plotted on a graph with price on the vertical axis and quantity on the horizontal axis.

2. Demand: This refers to the quantity of a good or service that consumers are willing and able to purchase at various price points.
   - The law of demand states that as the price of a good increases, the quantity demanded decreases (and vice versa).
   - This creates a downward-sloping demand curve on the same type of graph.

3. Market Equilibrium: This occurs at the intersection of the supply and demand curves.
   - At this point, the quantity that producers want to supply exactly equals the quantity that consumers want to buy.
   - The price at this intersection is called the equilibrium price, and the quantity is called the equilibrium quantity.

4. Price Mechanism: When markets are not in equilibrium, the price acts as a signal:
   - If price is above equilibrium, there's a surplus (excess supply), which puts downward pressure on prices.
   - If price is below equilibrium, there's a shortage (excess demand), which puts upward pressure on prices.

5. Shifts in Supply and Demand: Various factors can cause entire curves to shift:
   - Supply shifters include technology, input costs, number of sellers, and expectations.
   - Demand shifters include income, preferences, number of buyers, and expectations.

These principles help explain how markets allocate resources efficiently and how changes in market conditions affect prices and quantities.

Does this explanation help? Would you like me to elaborate on any specific aspect of supply and demand?
```

This output illustrates a successful tutoring interaction where a new session is created, and the AI responds to the student's economics question with a comprehensive explanation. The tutor's response demonstrates the system's ability to provide relevant, structured, and educational content, showcasing how the DeepSeek model can be effectively used for personalized academic support.

---

## Summary and Next Steps

In this lesson, we explored the `TutorService` class and its role in integrating the DeepSeek language model with tutoring sessions. We learned how to set up the class, load the system prompt, create sessions, and process student queries. The service layer is a vital component of our personal tutor application, ensuring that student interactions are handled effectively and that educational content is delivered in a clear and engaging manner.

As you move on to the practice exercises, take the opportunity to experiment with the `TutorService` functionality. This hands-on practice will reinforce the concepts covered in this lesson and prepare you for the next steps in our course. Keep up the great work, and I look forward to seeing your progress!

---


## Building the Tutor Service Layer

Implement the create_session method within the TutorService class.

Use the uuid module to generate a unique session ID.
Utilize the SessionManager to create a tutoring session, associating it with the student ID, session ID, and system prompt.
Ensure the method returns the generated session ID.
Test the session creation using the service.

Initialize an instance of TutorService.
Define a sample student ID to simulate a student.
Call the create_session method with the student ID and store the returned session ID.
Print the session ID to verify the successful creation of the tutoring session.
This task will guide you through the process of creating and managing tutoring sessions, a fundamental aspect of building a personal tutor service.

```python
import uuid
from models.session import SessionManager


class TutorService:
    def __init__(self):
        # TODO: Initialize SessionManager to handle session data
        self.session_manager = ________
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    # TODO: Define the create_session method
    # - Parameters: student_id
    # - Generate a unique session ID using uuid
    # - Use session_manager to create a session with student_id, session_id, and system_prompt
    # - Return the session_id


from services.tutor_service import TutorService

# TODO: Initialize a TutorService instance

# TODO: Define a variable with a sample student identifier, e.g., "student123"

# TODO: Call the create_session method with the student ID and store the session ID

# TODO: Print the session ID

```

Here’s the **implementation for `create_session`** within `TutorService` plus the code to test session creation.
Just follow the comments and replace the `TODO` lines as shown below!

---

```python
import uuid
from models.session import SessionManager


class TutorService:
    def __init__(self):
        # Initialize SessionManager to handle session data
        self.session_manager = SessionManager()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful tutor."

    def create_session(self, student_id: str) -> str:
        """Create a new tutoring session."""
        session_id = str(uuid.uuid4())
        self.session_manager.create_session(student_id, session_id, self.system_prompt)
        return session_id
```

---

### **Testing Session Creation**

```python
from services.tutor_service import TutorService

# Initialize a TutorService instance
tutor_service = TutorService()

# Define a variable with a sample student identifier
student_id = "student123"

# Call the create_session method with the student ID and store the session ID
session_id = tutor_service.create_session(student_id)

# Print the session ID
print(f"Session created with ID: {session_id}")
```

---

**Expected Output Example:**

```
Session created with ID: 123e4567-e89b-12d3-a456-426614174000
```

You can now **create and verify tutoring sessions**!
Let me know if you want to see how to check the session data or proceed with adding messages.


## Implementing Session Creation in the Tutor Service

## Implementing Query Processing for Tutor Service

## Implement AI Response Generation with DeepSeek in TutorService

## Testing Context Retention in AI Tutor Conversations

## Creating Multiple Tutoring Sessions for a Student